In [1]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable 
import numpy as np 
import tqdm 
from torch.utils import data 
import librosa
from scipy.io import wavfile
import json
import random
from scipy.signal import spectrogram

In [2]:
# Read in index for train, validation, test
with open('partition.json') as f:
    partition = json.load(f)

In [3]:
# constants used for normalizing (in pixel coordinate)
com_traj_minx = -1476
com_traj_maxx = 1758
com_traj_miny = -678
com_traj_maxy = 1785

In [4]:
class Dataset(data.Dataset):
    
    def __init__(self, list_IDs, transform=None):
        #'Initialization'
        self.list_IDs = list_IDs
        self.transform = transform
        
    def __len__(self):
        return len(self.list_IDs)
    
    def __getitem__(self, index):
        # Generates one sample of data 
        ID = self.list_IDs[index]
        filename = 'dataset_archive/audio/audio' + str(ID) + '.wav'
        srate, audio = wavfile.read(filename)
        audio = np.array(audio)
        
        # Audio Feature Extraction
        for i in range(0,7):
            freq,t,phase = spectrogram(audio[:,i], nperseg=512 , noverlap=256, mode='phase')
            freq,t,mag = spectrogram(audio[:,i], nperseg=512, noverlap=256, mode='magnitude')
            if (i == 0):
                audio_phase = torch.from_numpy(phase).unsqueeze(2).float()
                audio_mag = torch.from_numpy(mag).unsqueeze(2).float()
            else:
                audio_phase = torch.cat((audio_phase, torch.from_numpy(phase).unsqueeze(2).float()),2)
                audio_mag = torch.cat((audio_mag, torch.from_numpy(mag).unsqueeze(2).float()),2)
        audio_img = torch.cat((audio_mag, audio_phase), 2)
        audio_img = audio_img.permute(1,0,2)
        
        # Process complete trajectory and partially observed trajectory
        complete_traj = np.load("dataset_archive/trajectory/traj"+str(ID)+".npy")
        complete_traj = (complete_traj[0] - complete_traj).astype(float)
        obs_traj = np.load("dataset_archive/obs_traj/obs_traj"+str(ID)+".npy") 
        obs_traj = (obs_traj[0] - obs_traj).astype(float)
        obs_traj[:,0] = (obs_traj[:,0] - com_traj_minx) / (com_traj_maxx - com_traj_minx)
        obs_traj[:,1] = (obs_traj[:,1] - com_traj_miny) / (com_traj_maxy - com_traj_miny)
        complete_traj[:,0] = (complete_traj[:,0] - com_traj_minx) / (com_traj_maxx - com_traj_minx)
        complete_traj[:,1] = (complete_traj[:,1] - com_traj_miny) / (com_traj_maxy - com_traj_miny)
    
        if complete_traj.shape[0] > 135:
            complete_traj = complete_traj[0:135,:]
        if complete_traj.shape[0] < 135:
            for i in range(0, 135 - complete_traj.shape[0]):
                complete_traj = np.append(complete_traj, complete_traj[-1].reshape(1,2), axis=0)    

        complete_traj = torch.from_numpy(complete_traj).type(torch.FloatTensor)
        obs_traj = torch.from_numpy(obs_traj).type(torch.FloatTensor)

        return (audio_img, obs_traj), complete_traj

In [5]:
class Args(object):
    def __init__(self, name="Net", batch_size=4, test_batch_size=50,
                epochs=30, lr=1e-4, optimizer='Adam', momentum=0.9, weight_decay=0.0,
                seed=0, cuda = True):
        self.name = name;
        self.epochs = epochs
        self.batch_size = batch_size
        self.lr = lr
        self.weight_decay = weight_decay
        self.optimizer = optimizer
        self.momentum = momentum
        self.seed = seed
        self.cuda = cuda and torch.cuda.is_available()

In [6]:
args = Args()
params = {'batch_size': args.batch_size,
          'shuffle': True,
          'num_workers': 4}
# Generators
training_set = Dataset(partition['train'])
training_generator = data.DataLoader(training_set, **params)

validation_set = Dataset(partition['val'])
validation_generator = data.DataLoader(validation_set, **params)

In [7]:
class Encoder(nn.Module):
    """Bidirectional GRU"""
    def __init__(
        self, embedding_dim=64, h_dim=64, num_layers=1,
        dropout=0.0, bidirectional = True
    ):
        super(Encoder, self).__init__()

        self.hidden_dim = h_dim
        self.embedding_dim = embedding_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        self.encoder = nn.GRU(
            embedding_dim, h_dim, num_layers, dropout=dropout, bidirectional=bidirectional
        )

        self.spatial_embedding = nn.Linear(64, embedding_dim)

    def init_hidden(self, batch_size):
        if not self.bidirectional:
            return torch.zeros(self.num_layers, batch_size, self.hidden_dim).cuda()
        return torch.zeros(self.num_layers*2, batch_size, self.hidden_dim).cuda()

    def forward(self, obs_traj):
        """
        Inputs:
        - obs_traj: Tensor of shape (obs_len, batch, 2)
        """
        # Encode observed Trajectory
        batch_size = obs_traj.size(1)
        state_tuple = self.init_hidden(batch_size)
        output, state = self.encoder(obs_traj, state_tuple)
        final_h = state[0]
        return output, final_h

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cn1 = nn.Conv3d(in_channels=1, out_channels=64, kernel_size=(5,5,14), padding=(1,1,3))
        self.batchNorm1 = nn.BatchNorm3d(64)
        self.pooling1 = nn.MaxPool3d(kernel_size = (1,8,1), stride=(1,4,1))
        self.cn2 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(5,5,7), padding=(1,1,2))
        self.batchNorm2 = nn.BatchNorm3d(64)
        self.pooling2 = nn.MaxPool3d(kernel_size = (1,8,1), stride=(1,4,1))
        self.cn3 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(3,3,5), padding=(1,1,1))
        self.batchNorm3 = nn.BatchNorm3d(64)
        self.pooling3 = nn.MaxPool3d(kernel_size = (1,4,1), stride=(1,2,1))
        self.cn4 = nn.Conv3d(in_channels=64, out_channels=64, kernel_size=(3,3,3), padding=(1,0,0))
        self.batchNorm4 = nn.BatchNorm3d(64)
        self.pooling4 = nn.MaxPool3d(kernel_size = (1,4,1), stride=(1,2,1))
        
        self.biGRU1 = Encoder(
            embedding_dim=64,
            h_dim=64,
            num_layers=1,
            dropout=0
        )
        
        self.ln = nn.Linear(128, 64)
        
        self.biGRU2 = Encoder(
            embedding_dim=64,
            h_dim=64,
            num_layers=1,
            dropout=0
        )
        
        self.vision_encoder1 = nn.Linear(65, 256)
        self.vision_encoder2 = nn.Linear(256, 555)
        self.vision_encoder3 = nn.Linear(2, 128)
        
        
        
        self.fusion_module1 = nn.Linear(555, 256)
        self.fusion_module2 = nn.Linear(256, 135)
        
        self.traj_predictor1 = nn.Linear(256, 64)
        self.dropout = nn.Dropout(0.2)
        self.traj_predictor2 = nn.Linear(64, 16)
        self.dropout1 = nn.Dropout(0.1)
        self.traj_predictor3 = nn.Linear(16, 2)

        
    def forward(self, x, obs):
        
        obs = obs.permute(0,2,1)
        obs = F.relu(self.vision_encoder1(obs))
        obs = F.relu(self.vision_encoder2(obs))
        obs = obs.permute(0,2,1)
        obs = F.relu(self.vision_encoder3(obs))
        
        x = x.unsqueeze(1)
        x = F.relu(self.cn1(x))
        x = self.batchNorm1(x)
        x = self.pooling1(x)
        x = F.relu(self.cn2(x))
        x = self.batchNorm2(x)
        x = self.pooling2(x)
        x = F.relu(self.cn3(x))
        x = self.batchNorm3(x)
        x = self.pooling3(x)
        x = F.relu(self.cn4(x))
        x = self.batchNorm4(x)
        x = self.pooling4(x)
        
        x = x.squeeze(4)
        x = x.squeeze(3)
        x = x.permute(2,0,1)
        x, _ = self.biGRU1(x)
        x = self.ln(x)
        x = torch.tanh(x)
        x, _ = self.biGRU2(x)
        x = torch.tanh(x)
        
        x = x.permute(1,2,0)
        obs = obs.permute(0,2,1)
        x = torch.cat((x, obs),1)
        
        
        x = F.relu(self.fusion_module1(x))
        x = self.fusion_module2(x)
        
        x = x.permute(0,2,1)
        x = F.relu(self.traj_predictor1(x))
        x = self.dropout(x)
        x = F.relu(self.traj_predictor2(x))
        x = self.dropout1(x)
        x = torch.sigmoid(self.traj_predictor3(x))
        return x[:,:,0], x[:,:,1]

In [9]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
if args.cuda:
    model = model.cuda()
train_losses, train_accs = [], []
val_losses, val_accs = [], []
min_test_loss = 10000.
for epoch in range(args.epochs):
                
    # Training
    print("epoch " + str(epoch))
    model.train()
    
    total_loss, total_acc = 0., 0.
    progress_bar = tqdm.tqdm(training_generator, desc='Training')
    
    for batch_idx, (data, target) in enumerate(progress_bar):
        total_loss, total_acc = 0., 0.
        obs_traj = data[1].float()
        data = data[0].float()
        target = target.float()
        
        if args.cuda:
            data,obs_traj,target = data.cuda(), obs_traj.cuda(), target.cuda()
        data,obs_traj, target = Variable(data), Variable(obs_traj), Variable(target)

        
        optimizer.zero_grad()
        output = model(data, obs_traj)
        loss_x = F.mse_loss(output[0], target[:,:,0], reduction="sum") 
        loss_y = F.mse_loss(output[1], target[:,:,1], reduction="sum") 
        loss = loss_x + loss_y
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.data.item())
        total_loss += loss.data
        progress_bar.clear()
        progress_bar.set_description('Epoch: {} loss: {:.4f}'.format(epoch, total_loss / (batch_idx + 1)))
        progress_bar.refresh()
    
    
    # Validation
    model.eval()
    test_loss, acc = 0., 0.
    progress_bar = tqdm.tqdm(validation_generator, desc='Validation')
    with torch.no_grad():
        for data, target in progress_bar:
            obs_traj = data[1].float()
            data = data[0].float()
            target = target.float()
            if args.cuda:
                data,obs_traj,target = data.cuda(), obs_traj.cuda(), target.cuda()
            data, obs_traj, target = Variable(data),Variable(obs_traj), Variable(target)
            data = data.float()
            output = model(data, obs_traj)
            loss_x = F.mse_loss(output[0], target[:,:,0], reduction="sum")  
            loss_y = F.mse_loss(output[1], target[:,:,1], reduction="sum")   # sum up batch loss
            test_loss += loss_x + loss_y
    test_loss /= len(partition['val'])

    val_losses.append(test_loss.item())
    if epoch % 10 == 0 and epoch != 0:
        args.lr /= 5
        for param_group in optimizer.param_groups:
            param_group['lr'] = args.lr
            print(param_group['lr'])
    progress_bar.clear()
    progress_bar.write(
        '\nEpoch: {} validation test results - Average val_loss: {:.4f}'.format(
            epoch, test_loss))
    if (test_loss < min_test_loss):
        torch.save(model, "combined_checkpoint_new.pkl")
        min_test_loss = test_loss

Training:   0%|          | 0/288 [00:00<?, ?it/s]

epoch 0


Validation: 100%|██████████| 32/32 [00:09<00:00,  3.38it/s]
/home/frankbu/.local/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/frankbu/.local/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
Training:   0%|          | 0/288 [00:00<?, ?it/s]


Epoch: 0 validation test results - Average val_loss: 8.4167
epoch 1


Training:   0%|          | 0/288 [00:00<?, ?it/s]

...


Epoch: 27 validation test results - Average val_loss: 1.3423
epoch 28


Training:   0%|          | 0/288 [00:00<?, ?it/s]


Epoch: 28 validation test results - Average val_loss: 1.3759
epoch 29


Validation: 100%|██████████| 32/32 [00:09<00:00,  3.38it/s]


Epoch: 29 validation test results - Average val_loss: 1.3767
